# Welcom to EarthStat

In [38]:
# !pip install earthstat

In [35]:
from earthstat.earthstat import EarthStat

In [3]:
# predictor_name              = 'FPAR' # Predictor Name
# predictor_dir               = 'FPAR' # Predictor Dir
# mask_path                   = 'crop mask/Percent_Maize.tif' # Mask Path
# shapefile_path              = 'shapefile/test_filtered_gaul1_asap.shp' # Shapefile Path
# selected_countries          = ["Norway", "Spain"]	   # Choose Name Of Countries
# country_column_name         = 'adm0_name'		   # Name of Column contains countries 
# invalid_values              =[255, 254, 251] # Set None if no invalid Values

In [5]:
# aggregate_fpar = EarthStat(predictor_name)

In [6]:
# aggregate_fpar.initDataDir(predictor_dir)

Predictor Summary:

predictor: FPAR
total_tiff_files: 2
date_range: 2015-07-21 to 2015-08-01
directory: FPAR
CRS: WGS 84
Extent: BoundingBox(left=-180.004464285715, bottom=-56.00446430667502, right=179.99553577188502, top=75.004464285715)
Data Type: uint8
NoData Value: 255.0
Spatial Resolution: 80640x29346
Pixel Size: (0.004464285715, 0.004464285715)

Predictor Paths Initialized Correctly, Initialize The Mask's Path


In [7]:
# aggregate_fpar.initMaskPath(mask_path)

Mask Summary:

Mask_path: crop mask/Percent_Maize.tif
CRS: WGS 84
Extent: BoundingBox(left=-180.0, bottom=-89.99999999999994, right=180.0, top=90.00000000000006)
Data Type: uint16
NoData Value: 99.0
Spatial Resolution: (7200, 3600)
Pixel Size: (0.05, 0.05)
Min/Max Value: (0, 10000)

Mask Initialized Correctly, Initialize The Shapefile


In [8]:
# aggregate_fpar.initShapefilePath(shapefile_path)

Shapefile Summary:

Geometry Type: ['MultiPolygon' 'Polygon']
Coordinate Reference System (CRS): WGS_84_Pseudo-Mercator
Extent: [-2022634.81443587  3203512.94054597  3458167.43087574 11466900.28122624]
Feature Count: 36
Attributes: ['adm1_code', 'adm1_name', 'adm0_code', 'adm0_name', 'adm0_name_', 'adm1_name_', 'asap1_id', 'asap0_id', 'geometry']

Shapefile Initialized Correctly, You Can Check The Data Compatibility


In [9]:
# aggregate_fpar.DataCompatibility()

ISSUE: PIXEL SIZE MISMATCH
############################################################
Mask Pixel Size        : (0.05, 0.05)
Predicotr Pixel Size   : (0.004464285715, 0.004464285715)
############################################################

NO ISSUE: The projections of the mask and predictor are identical: WGS 84
ISSUE: The projections of the raster data and shapefile are not identical
############################################################
raster data CRS: WGS 84
shapefile CRS: WGS_84_Pseudo-Mercator
############################################################


COMPATIBILITY ISSUE DETECTED: The data is not compatible based on the current checks.


In [10]:
# # Resampling Methods [nearest, bilinear, cubic, average]
# aggregate_fpar.fixCompatibilityIssues(rescale_factor=None, # None = Rescale OFF
#                                       resampling_method="bilinear") # Defualt Bilinear

Checking for compatibility issues...
Compatibility issues detected. Starting the fix process...
- Resampling the mask to match the raster's resolution and extent.
- Reprojecting the shapefile to match the raster's coordinate reference system.
Resampling mask...
Filtered shapefile saved to: crop mask/rescaled_resampled_average_Percent_Maize.tif

Reprojecting shapefile...
Shapefile reprojected to match raster CRS and saved to shapefile/reprojected_test_filtered_gaul1_asap.shp
Mask resampled successfully. Updated mask path: [crop mask/rescaled_resampled_average_Percent_Maize.tif]

Re-checking data compatibility after applying fixes...

NO ISSUE: The pixel sizes of the mask and predictor are identical: (0.004464285715, 0.004464285715)
NO ISSUE: The projections of the mask and predictor are identical: WGS 84
NO ISSUE: The projections of the raster data and shapefile are identical: WGS 84

All compatibility issues have been resolved. Data is now compatible.


In [11]:
# aggregate_fpar.selectRegionOfInterest(selected_countries,
#                                       country_column_name)

Filtered shapefile saved to: shapefile/filtered_reprojected_test_filtered_gaul1_asap.shp
Region of Interest (ROI) successfully selected based on the specified countries: Norway, Spain.


In [12]:
# Running clip withtout select ROI, will clip using main shapefile
# aggregate_fpar.clipPredictor()

Clipping the predictor data...


Clipping Rasters: 100%|██████████| 2/2 [00:08<00:00,  4.01s/it]

Clipping operation successful with the Region of Interest (ROI).


In [13]:
# aggregate_fpar.runAggregation()

Starting aggregation...
Starting aggregation with the selected Region of Interest (ROI) for FPAR.


Processing rasters: 100%|██████████| 2/2 [00:07<00:00,  3.80s/raster]

Aggregation complete. Data saved to Aggregated_FPAR.csv.
